In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!ls '/content/drive/My Drive/Data Analysis'


In [ ]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/Data Analysis/Data sets/Superstore.xls')
df.head(4)

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.strip()  
df.dropna(inplace=True)

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df.reset_index(drop=True, inplace=True)


In [ ]:
df["Outlier"] = (df['Sales'] >= 0) & (df['Quantity'] > 0) & (df['Discount'] >= 0) & (df['Profit'] >= 0) 
df["Outlier"].head(2)

In [ ]:
df['Invalid Data'] = df['Discount'] <= 1 
df['Invalid Data'].head(2)

In [ ]:
df['Postal Code'] = df['Postal Code'].astype('Int64').astype(str)
df['Postal Code'].dtypes


In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y',errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y',errors='coerce')
df.info()


In [ ]:
df['Year'] = df['Order Date'].dt.year 
df['Month'] = df['Order Date'].dt.month 
df['Month Name'] = df['Order Date'].dt.strftime('%B')
df['Weekday'] = df['Order Date'].dt.day_name() 

In [ ]:
df['Year'].head(2)

In [ ]:
df['Month'].head(2)

In [ ]:
df['Month Name'].head(2)

In [ ]:
df['Weekday'].head(2)

In [ ]:
df['Profit_Margin'] = df['Profit'] / df['Sales']
df['Profit_Margin'] = df['Profit_Margin'].fillna(0) 
df['Profit_Margin'].head(2)

In [ ]:
df['Profit_Margin_Percent'] = df['Profit_Margin'] * 100
df['Profit_Margin_Percent'].round(2) .head(2)

In [ ]:
df['Revenue_After_Discount'] = df['Sales'] * (1 - df['Discount'])
df['Revenue_After_Discount'].head(2)

In [ ]:
print(df['Ship Mode'].unique())
print(df['Segment'].unique())
print(df['Region'].unique())
print(df['Category'].unique())
print(df['Sub-Category'].unique())


In [ ]:
df.describe()

In [ ]:
df.describe().columns

In [ ]:
df.describe().to_csv('describe_output.csv')

In [ ]:
numeric_df = df.select_dtypes(include=np.number)
corr = numeric_df.corr()
corr

In [ ]:
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(numeric_df.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix')

plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

**Which products generate the most revenue, and which are underperforming?**

In [ ]:
top_products = df.groupby('Product Name')['Sales'].sum().sort_values(ascending=False).head(10)
bottom_products = df.groupby('Product Name')['Sales'].sum().sort_values().head(10)



In [ ]:
top_products.head(10)

In [ ]:
bottom_products.tail(10)

In [ ]:
#Top 10 Products by Revenue
top10 = top_products.head(10)

plt.figure(figsize=(12,6))
ax = sns.barplot(x=top10.values, y=top10.index, palette='pastel')
ax.xaxis.grid(True, linestyle='--', alpha=0.7)
ax.set_axisbelow(True)


for i, value in enumerate(top10.values):
    plt.text(value + 50, i, f"${value:,.0f}", va='center', fontsize=10)

plt.title("Top 10 Products by Revenue")
plt.xlabel("Total Revenue")
plt.ylabel("Product Name")
plt.tight_layout(pad=3.0)
plt.show()

**The Canon imageCLASS 2200 Advanced Copier** generates the most revenue by far, earning over $60,000, making it the top-performing product.

Other high-revenue products include:

**Fellowes PB500 Electric Punch Binding Machine**

**Cisco TelePresence System**

**HON 5400 Series Task Chairs**

These all have strong sales in the **$20k–$30k range.**

On the other hand, products like:

**High Speed Automatic Electric Letter Opener**

**GBC DocuBind P400 Electric Binding System**

are among the lowest in revenue within the top 10 list, each generating under **$20,000**, indicating they are underperforming compared to others.

In [ ]:
plt.figure(figsize=(14,8))
ax = sns.barplot(x=bottom_products.values, y=bottom_products.index, palette='pastel')

ax.xaxis.grid(True, linestyle='--', alpha=0.7)
ax.set_axisbelow(True)

for i, value in enumerate(bottom_products.values):
    plt.text(value, i + 0.1, f"${value:,.0f}", ha='center', fontsize=10)


plt.title("Bottom 10 Products by Revenue (Underperforming)")
plt.xlabel("Total Revenue")
plt.ylabel("Product Name")
plt.tight_layout(pad=3.0)
plt.show()

**The Newell 308** generates **the least revenue**, with sales just over 8 units, making it the lowest-performing product in terms of revenue.

Other low-revenue products include:

**Stockwell Gold Paper Clips**

**4009 Highlighters**

**Xerox 1989**

These all have weak sales in the **7–8 range**.

On the other hand, products like:

**Avery 5**

Eureka Disposable Bags for Sanitaire Vibra Groomer I Upright Vac

are among the **lowest in revenue** within the bottom 10 list, each generating **under 6 sales**, indicating they are **severely underperforming** compared to others.

In [ ]:
!pip install dash dash-daq pandas plotly


**Plotyly Dashboard**

In [ ]:
import plotly.express as px
import dash
from dash import dcc, html
import dash_daq as daq


product_sales = df.groupby('Product Name', as_index=False)['Sales'].sum()
top10 = product_sales.sort_values('Sales', ascending=False).head(10)
bottom10 = product_sales.sort_values('Sales', ascending=True).head(10)

total_revenue = df['Sales'].sum()
total_products = df['Product Name'].nunique()
avg_revenue = total_revenue / total_products


app = dash.Dash(__name__)
app.title = 'Product Revenue Dashboard'


app.layout = html.Div(style={'backgroundColor': '#F9F9F9', 'color': '#222222', 'padding': '20px'}, children=[
    html.H1('Product Revenue Dashboard', style={'textAlign': 'center', 'color': '#333333'}),

    
    html.Div([
        html.H2('Key Metrics', style={'textAlign': 'center', 'color': '#444444'}),
        html.Div([
            daq.LEDDisplay(
                label="Total Revenue ($)",
                value=f"{int(total_revenue):,}",
                size=40,
                color="#00796B",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
            daq.LEDDisplay(
                label="Total Unique Products",
                value=str(total_products),
                size=40,
                color="#F57C00",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
            daq.LEDDisplay(
                label="Avg Revenue/Product ($)",
                value=f"{int(avg_revenue):,}",
                size=40,
                color="#0288D1",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'padding': '20px'})
    ]),

    # Top 10 Products Graph
    html.Div([
        html.H2('Top 10 Products by Revenue', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.bar(top10, x='Sales', y='Product Name',
                          orientation='h', color='Sales',
                          color_continuous_scale='Viridis',
                          title='Top 10 Products by Revenue')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(
                    showgrid=True,
                    gridcolor='rgba(0,0,0,0.1)',
                    zeroline=False
                ),
                yaxis=dict(
                    showgrid=False
                ),
                margin=dict(l=150, r=40, t=60, b=40),
                yaxis_autorange='reversed' 
            )
        )
    ]),

    # Bottom 10 Products Graph
    html.Div([
        html.H2('Bottom 10 Underperforming Products', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.bar(bottom10, x='Sales', y='Product Name',
                          orientation='h', color='Sales',
                          color_continuous_scale='Reds',
                          title='Bottom 10 Products by Revenue')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(
                    showgrid=True,
                    gridcolor='rgba(0,0,0,0.1)',
                    zeroline=False
                ),
                yaxis=dict(
                    showgrid=False
                ),
                margin=dict(l=150, r=40, t=60, b=40),
                yaxis_autorange='reversed'
            )
        )
    ])
])

if __name__ == '__main__':
    app.run(debug=True)


**Are certain regions or states more profitable than others?**

In [ ]:
profit_by_state = df.groupby('State')['Profit'].sum().sort_values(ascending=False)
profit_by_region = df.groupby('Region')['Profit'].sum().sort_values(ascending=False)


In [ ]:
profit_by_state.head(5)

In [ ]:
profit_by_region.head(5)

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x=profit_by_region.values, y=profit_by_region.index, palette='pastel')
plt.title('Total Profit by Region')
plt.xlabel('Profit')
plt.ylabel('Region')
plt.grid(axis='x', linestyle='--', alpha=0.5)


for i, v in enumerate(profit_by_region.values):
    ax.text(v, i, f"${v:,.2f}", va='center', ha='left', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
#Top 10 Most Profitable States
top10_states = profit_by_state.head(10)
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=top10_states.values, y=top10_states.index, palette='pastel')
plt.title('Top 10 Most Profitable States')
plt.xlabel('Profit')
plt.ylabel('State')
plt.grid(axis='x', linestyle='--', alpha=0.5)

for i, v in enumerate(top10_states.values):
    ax.text(v, i, f"${v:,.2f}", va='center', ha='left', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

**California** generates the most profit by far, earning **over $75,000**, making it the top-performing state in terms of profitability.

Other high-profit states include:

**New York**

**Washington**

Michigan **bold text**

These all have strong profits in the **$20k–$70k** range.

On the other hand, states like:

**Oregon**

Florida

are among the least profitable regions, each generating losses of over **-$5,000**, indicating they are underperforming compared to others

In [ ]:
# Bottom 10 Least Profitable States
bottom10_states = profit_by_state.tail(10)

plt.figure(figsize=(10, 6))
ax = sns.barplot(x=bottom10_states.values, y=bottom10_states.index, palette='pastel') 
plt.title('Bottom 10 Least Profitable States')
plt.xlabel('Profit')
plt.ylabel('State')
plt.grid(axis='x', linestyle='--', alpha=0.5)


for i, v in enumerate(bottom10_states.values): 
    ax.text(v, i, f"${v:,.2f}", va='center', ha='left', fontweight='bold', fontsize=10)


plt.tight_layout()
plt.show()

**Texas** generates the largest losses by far, losing over **$25,000**, making it the least profitable state overall.

Other low-profit states include:

**Ohio**

**Pennsylvania**

**Illinois**

These all have significant losses in the **$15k–$20k** range.

On the other hand, states like:

**Oregon**

**Florida**

are among the least profitable within the bottom 10 list, each generating losses under **$5,000**, indicating they are slightly better performing compared to others in this group.

In [ ]:
#Data Aggregation 
profit_by_state = df.groupby('State', as_index=False)['Profit'].sum().sort_values('Profit', ascending=False)

top10_states = profit_by_state.head(10)
bottom10_states = profit_by_state.tail(10)

total_profit = df['Profit'].sum()
total_states = df['State'].nunique()
avg_profit_per_state = total_profit / total_states


app = dash.Dash(__name__)
app.title = 'State Profitability Dashboard'

app.layout = html.Div(style={'backgroundColor': '#F9F9F9', 'color': '#222222', 'padding': '20px'}, children=[
    html.H1('State-wise Profitability Dashboard', style={'textAlign': 'center', 'color': '#333333'}),

   
    html.Div([
        html.H2('Key Profit Metrics', style={'textAlign': 'center', 'color': '#444444'}),
        html.Div([
            daq.LEDDisplay(
                label="Total Profit ($)",
                value=f"{int(total_profit):,}",
                size=40,
                color="#43A047",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
            daq.LEDDisplay(
                label="Number of States",
                value=str(total_states),
                size=40,
                color="#FBC02D",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
            daq.LEDDisplay(
                label="Avg Profit/State ($)",
                value=f"{int(avg_profit_per_state):,}",
                size=40,
                color="#1E88E5",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'padding': '20px'})
    ]),

    # Top 10 Most Profitable States
    html.Div([
        html.H2('Top 10 Most Profitable States', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.bar(top10_states, x='Profit', y='State',
                          orientation='h', color='Profit',
                          color_continuous_scale='Greens',
                          title='Top 10 Profitable States')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)', zeroline=False),
                yaxis=dict(showgrid=False),
                margin=dict(l=150, r=40, t=60, b=40),
                yaxis_autorange='reversed'
            )
        )
    ]),

    # Bottom 10 Least Profitable States
    html.Div([
        html.H2('Bottom 10 Least Profitable States', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.bar(bottom10_states, x='Profit', y='State',
                          orientation='h', color='Profit',
                          color_continuous_scale='Reds',
                          title='Bottom 10 Least Profitable States')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)', zeroline=False),
                yaxis=dict(showgrid=False),
                margin=dict(l=150, r=40, t=60, b=40),
                yaxis_autorange='reversed'
            )
        )
    ])
])

if __name__ == '__main__':
    app.run(debug=True)

**When do sales peak or dip, and what seasonal trends exist?**

In [ ]:

df['Month_Year'] = df['Order Date'].dt.to_period('M').astype(str)
monthly_sales = df.groupby('Month_Year')['Sales'].sum().reset_index()
monthly_sales.head(2)

In [ ]:
monthly_sales['Month_Year'] = pd.to_datetime(monthly_sales['Month_Year'])
monthly_sales = monthly_sales.sort_values('Month_Year')

plt.figure(figsize=(14, 6))
ax = sns.lineplot(data=monthly_sales, x='Month_Year', y='Sales', marker='o', color='skyblue', linewidth=2)


plt.grid(axis='y', linestyle='--', alpha=0.6)


for i in range(len(monthly_sales)):
    plt.text(monthly_sales['Month_Year'].iloc[i],
             monthly_sales['Sales'].iloc[i] + 500,
             f"${monthly_sales['Sales'].iloc[i]:,.0f}",
             ha='center', fontsize=8)

plt.title('Monthly Sales Trend Over Time')
plt.xlabel('Month-Year')
plt.ylabel('Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
#Average sales by calendar month (seasonality)
seasonal_sales = df.groupby('Month')['Sales'].mean().reset_index()

import calendar
seasonal_sales['Month Name'] = seasonal_sales['Month'].apply(lambda x: calendar.month_abbr[x])

plt.figure(figsize=(12, 5))
ax = sns.barplot(x='Month Name', y='Sales', data=seasonal_sales, palette='pastel')
plt.grid(axis='y', linestyle='--', alpha=0.6)


for i, v in enumerate(seasonal_sales['Sales']):
    plt.text(i, v + 200, f"${v:,.0f}", ha='center', fontweight='bold', fontsize=10)

plt.title('Average Sales by Month (Seasonal Trend)')
plt.xlabel('Month')
plt.ylabel('Average Sales')
plt.tight_layout()
plt.show()


In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Month'] = df['Order Date'].dt.month
df['Year'] = df['Order Date'].dt.year
df['Month_Year'] = df['Order Date'].dt.to_period('M').astype(str)

# Monthly sales trend
monthly_sales = df.groupby('Month_Year', as_index=False)['Sales'].sum()
monthly_sales['Month_Year'] = pd.to_datetime(monthly_sales['Month_Year'])
monthly_sales = monthly_sales.sort_values('Month_Year')

# Seasonal average by calendar month
seasonal_sales = df.groupby('Month', as_index=False)['Sales'].mean()
import calendar
seasonal_sales['Month Name'] = seasonal_sales['Month'].apply(lambda x: calendar.month_abbr[x])
seasonal_sales = seasonal_sales.sort_values('Month')


total_sales = df['Sales'].sum()
avg_monthly_sales = monthly_sales['Sales'].mean()
peak_month = monthly_sales.loc[monthly_sales['Sales'].idxmax(), 'Month_Year'].strftime('%b %Y')

app = dash.Dash(__name__)
app.title = 'Sales Trend & Seasonality Dashboard'


app.layout = html.Div(style={'backgroundColor': '#F9F9F9', 'color': '#222222', 'padding': '20px'}, children=[
    html.H1('Sales Trend & Seasonality Dashboard', style={'textAlign': 'center', 'color': '#333333'}),

    html.Div([
        html.H2('Sales KPIs', style={'textAlign': 'center', 'color': '#444444'}),
        html.Div([
            daq.LEDDisplay(
                label="Total Sales ($)",
                value=f"{int(total_sales):,}",
                size=40,
                color="#43A047",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
            daq.LEDDisplay(
                label="Avg Monthly Sales ($)",
                value=f"{int(avg_monthly_sales):,}",
                size=40,
                color="#1E88E5",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
            daq.LEDDisplay(
                label="Peak Month",
                value=peak_month,
                size=40,
                color="#F57C00",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'padding': '20px'})
    ]),

    # Monthly Trend Line Chart
    html.Div([
        html.H2('Monthly Sales Over Time', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.line(monthly_sales, x='Month_Year', y='Sales',
                           title='Monthly Sales Trend',
                           markers=True)
            .update_traces(line_color='skyblue')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(
                    title='Month-Year',
                    showgrid=True,
                    gridcolor='rgba(0,0,0,0.1)',
                    tickangle=45
                ),
                yaxis=dict(
                    title='Sales',
                    showgrid=True,
                    gridcolor='rgba(0,0,0,0.1)'
                ),
                margin=dict(l=40, r=40, t=60, b=60)
            )
        )
    ]),

    # Seasonal Bar Chart
    html.Div([
        html.H2('Average Sales by Month (Seasonality)', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.bar(seasonal_sales, x='Month Name', y='Sales',
                          color='Sales',
                          color_continuous_scale='Viridis', 
                          title='Average Sales by Calendar Month')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(
                    title='Month',
                    showgrid=False
                ),
                yaxis=dict(
                    title='Average Sales',
                    showgrid=True,
                    gridcolor='rgba(0,0,0,0.1)'
                ),
                margin=dict(l=40, r=40, t=60, b=60)
            )
        )
    ])
])

if __name__ == '__main__':
    app.run(debug=True)

**Which customer segments or categories contribute most to profits?**

In [ ]:
segment_profit = df.groupby('Segment')['Profit'].sum().sort_values(ascending=False)
category_profit = df.groupby('Category')['Profit'].sum().sort_values(ascending=False)


In [ ]:
segment_profit.head(4)

In [ ]:
category_profit.head()

In [ ]:

plt.figure(figsize=(8, 5))
ax = sns.barplot(x=segment_profit.values, y=segment_profit.index, palette="pastel")
ax.set_title('Profit Contribution by Customer Segment', fontsize=14)
ax.set_xlabel('Total Profit')
ax.set_ylabel('Segment')


for i, v in enumerate(segment_profit.values):
    ax.text(v + 1000, i, f"${v:,.0f}", va='center', fontsize=10)

ax.xaxis.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
ax = sns.barplot(x=category_profit.values, y=category_profit.index,palette="pastel")
ax.set_title('Profit Contribution by Product Category', fontsize=14)
ax.set_xlabel('Total Profit')
ax.set_ylabel('Category')


for i, v in enumerate(category_profit.values):
    ax.text(v + 1000, i, f"${v:,.0f}", va='center', fontsize=10)


ax.xaxis.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
segment_profit = df.groupby('Segment', as_index=False)['Profit'].sum().sort_values(by='Profit', ascending=False)
category_profit = df.groupby('Category', as_index=False)['Profit'].sum().sort_values(by='Category', ascending=False)


top_segment = segment_profit.iloc[0]
top_category = category_profit.iloc[0]


app = dash.Dash(__name__)
app.title = 'Profit Contribution Dashboard'


app.layout = html.Div(style={'backgroundColor': '#F9F9F9', 'color': '#222222', 'padding': '20px'}, children=[
    html.H1('Profit Contribution Dashboard', style={'textAlign': 'center'}),

   
    html.Div([
        html.H2('Top Contributors', style={'textAlign': 'center', 'color': '#444444'}),
        html.Div([
            daq.LEDDisplay(
                label="Top Segment",
                value=top_segment['Segment'],
                size=40,
                color="#00897B",
                backgroundColor="#e0f2f1",
                style={'margin': '0 20px'}
            ),
            daq.LEDDisplay(
                label="Top Category",
                value=top_category['Category'],
                size=40,
                color="#3949AB",
                backgroundColor="#e8eaf6",
                style={'margin': '0 20px'}
            ),
        ], style={'display': 'flex', 'justifyContent': 'center', 'padding': '20px'})
    ]),

    html.Div([
        html.H2('Profit by Customer Segment', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.bar(segment_profit, x='Profit', y='Segment',
                          orientation='h',
                          color='Profit',
                          color_continuous_scale=px.colors.sequential.Blues,  
                          title='Total Profit by Segment')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(title='Profit', showgrid=True, gridcolor='rgba(0,0,0,0.1)'),
                yaxis=dict(title='Segment'),
                margin=dict(l=40, r=40, t=60, b=60)
            )
        )
    ]),

    html.Div([
        html.H2('Profit by Product Category', style={'color': '#333333'}),
        dcc.Graph(
            figure=px.bar(category_profit, x='Profit', y='Category',
                          orientation='h',
                          color='Profit',
                          color_continuous_scale=px.colors.sequential.Blues,  
                          title='Total Profit by Category')
            .update_layout(
                paper_bgcolor='#F9F9F9',
                plot_bgcolor='#F9F9F9',
                font_color='#222222',
                xaxis=dict(title='Profit', showgrid=True, gridcolor='rgba(0,0,0,0.1)'),
                yaxis=dict(title='Category'),
                margin=dict(l=40, r=40, t=60, b=60)
            )
        )
    ])
])

if __name__ == '__main__':
    app.run(debug=True)

**How does discounting impact profitability and sales volume?**

In [ ]:
# Bin the discounts
discount_bins = pd.cut(df['Discount'], bins=[0, 0.1, 0.2, 0.3, 0.5, 0.7], include_lowest=True)
grouped_discount = df.copy()
grouped_discount['Discount Range'] = discount_bins.astype(str)
avg_profit_discount = grouped_discount.groupby('Discount Range', as_index=False)['Profit'].mean()
avg_profit_discount.head(2)


In [ ]:
sns.set(style="whitegrid", palette="pastel")

#Discount vs Profit
plt.figure(figsize=(10, 6))
ax = sns.scatterplot(data=df, x='Discount', y='Profit', alpha=0.6)
sns.regplot(data=df, x='Discount', y='Profit', scatter=False, color='coral', line_kws={"lw": 2})

ax.set_title('Impact of Discount on Profit', fontsize=14)
ax.set_xlabel('Discount (%)')
ax.set_ylabel('Profit ($)')
ax.xaxis.grid(True, linestyle='--', alpha=0.7)
ax.yaxis.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
#Discount vs Sales
plt.figure(figsize=(10, 6))
ax = sns.scatterplot(data=df, x='Discount', y='Sales', alpha=0.6)
sns.regplot(data=df, x='Discount', y='Sales', scatter=False, color='blue', line_kws={"lw": 2})

ax.set_title('Impact of Discount on Sales Volume', fontsize=14)
ax.set_xlabel('Discount (%)')
ax.set_ylabel('Sales ($)')
ax.xaxis.grid(True, linestyle='--', alpha=0.7)
ax.yaxis.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Avg Profit & Sales per Discount Level
discount_bins = pd.cut(df['Discount'], bins=[0, 0.1, 0.2, 0.3, 0.5, 0.7], include_lowest=True)
grouped = df.groupby(discount_bins).agg({'Profit': 'mean', 'Sales': 'mean'}).reset_index()

#Average Profit per Discount Range
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=grouped, x='Discount', y='Profit')
ax.set_title('Average Profit per Discount Range', fontsize=14)
ax.set_xlabel('Discount Range')


ax.set_ylabel('Average Profit')
for i, v in enumerate(grouped['Profit']):
    ax.text(i, v + 5, f"${v:,.2f}", ha='center')
ax.yaxis.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
#Discounted Binned Data Preparation 
discount_bins = pd.cut(df['Discount'], bins=[0, 0.1, 0.2, 0.3, 0.5, 0.7], include_lowest=True)
grouped_discount = df.copy()
grouped_discount['Discount Range'] = discount_bins
avg_profit_discount = grouped_discount.groupby('Discount Range', as_index=False)['Profit'].mean()

#Scatter Plots with Trend Lines 
fig_discount_profit = px.scatter(df, x='Discount', y='Profit',
                                 title='Discount vs Profit',
                                 trendline='ols',
                                 color_discrete_sequence=['#42A5F5'])
fig_discount_profit.update_layout(
    paper_bgcolor='#F9F9F9',
    plot_bgcolor='#F9F9F9',
    font_color='#222222',
    xaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)'),
    yaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)'),
    margin=dict(l=40, r=40, t=60, b=60)
)

fig_discount_sales = px.scatter(df, x='Discount', y='Sales',
                                title='Discount vs Sales',
                                trendline='ols',
                                color_discrete_sequence=['#66BB6A'])
fig_discount_sales.update_layout(
    paper_bgcolor='#F9F9F9',
    plot_bgcolor='#F9F9F9',
    font_color='#222222',
    xaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)'),
    yaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)'),
    margin=dict(l=40, r=40, t=60, b=60)
)

#Average Profit per Discount Range 
fig_avg_profit_range = px.bar(avg_profit_discount, x='Discount Range', y='Profit',
                              title='Average Profit by Discount Range',
                              color='Profit',
                              color_continuous_scale=px.colors.sequential.Blues)
fig_avg_profit_range.update_layout(
    paper_bgcolor='#F9F9F9',
    plot_bgcolor='#F9F9F9',
    font_color='#222222',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)'),
    margin=dict(l=40, r=40, t=60, b=60)
)


app = dash.Dash(__name__)
app.title = 'Discount Impact Dashboard'


app.layout = html.Div(style={'backgroundColor': '#F9F9F9', 'color': '#222222', 'padding': '20px'}, children=[
    html.H1('Discount Impact on Profit & Sales', style={'textAlign': 'center'}),

    #Discount vs Profit 
    html.Div([
        html.H2('How Discounts Affect Profit', style={'color': '#333333'}),
        dcc.Graph(figure=fig_discount_profit)
    ]),
    
    #Discount vs Sales 
    html.Div([
        html.H2('How Discounts Affect Sales Volume', style={'color': '#333333'}),
        dcc.Graph(figure=fig_discount_sales)
    ]),

    #Avg Profit by Discount Range 
    html.Div([
        html.H2('Average Profit by Discount Range', style={'color': '#333333'}),
        dcc.Graph(figure=fig_avg_profit_range)
    ])
])


if __name__ == '__main__':
    app.run(debug=True)


**Is there inventory inefficiency or overstocking in low-performing items?**

In [ ]:
product_perf = df.groupby('Product Name').agg({'Sales': 'sum', 'Quantity': 'sum', 'Profit': 'sum'})
low_perf = product_perf[(product_perf['Sales'] < product_perf['Sales'].quantile(0.25)) &
                        (product_perf['Quantity'] > product_perf['Quantity'].quantile(0.75))]


In [ ]:
#Sorting for clear view 
low_perf_sorted = low_perf.sort_values('Quantity', ascending=False).head(20)  # top 20 for clarity

plt.figure(figsize=(14, 8))
ax = sns.barplot(data=low_perf_sorted, x='Quantity', y=low_perf_sorted.index, palette='pastel')

ax.xaxis.grid(True, linestyle='--', alpha=0.6)
plt.title('Potential Overstocked Low-Performing Products', fontsize=16)
plt.xlabel('Quantity Sold')
plt.ylabel('Product Name')
plt.tight_layout()
plt.show()

In [ ]:
#Data Aggregation 
product_perf = df.groupby('Product Name').agg({'Sales': 'sum', 'Quantity': 'sum', 'Profit': 'sum'}).reset_index()

#Identify Low-Performing High-Quantity Products 
low_perf = product_perf[
    (product_perf['Sales'] < product_perf['Sales'].quantile(0.25)) &
    (product_perf['Quantity'] > product_perf['Quantity'].quantile(0.75))
].sort_values('Quantity', ascending=False).head(20)  # Top 20 for clarity



#Bar Plot for Low-Performing High-Quantity Products 
fig_low_perf = px.bar(low_perf,
                      x='Quantity',
                      y='Product Name',
                      orientation='h',
                      title='Low Sales but High Quantity Products (Potential Overstock)',
                      color='Sales',
                      color_continuous_scale=px.colors.sequential.RdPu)

fig_low_perf.update_layout(
    paper_bgcolor='#F9F9F9',
    plot_bgcolor='#F9F9F9',
    font_color='#222222',
    xaxis=dict(showgrid=True, gridcolor='rgba(0,0,0,0.1)', title='Quantity Sold'),
    yaxis=dict(showgrid=False, title='Product Name'),
    margin=dict(l=100, r=40, t=60, b=60)
)


app = dash.Dash(__name__)
app.title = 'Inventory Inefficiency Dashboard'

app.layout = html.Div(style={'backgroundColor': '#F9F9F9', 'color': '#222222', 'padding': '20px'}, children=[
    html.H1('Inventory Efficiency Insights', style={'textAlign': 'center'}),

    html.Div([
        html.H2('High Quantity but Low Sales Products', style={'color': '#333333'}),
        dcc.Graph(figure=fig_low_perf)
    ])
])

if __name__ == '__main__':
    app.run(debug=True)


**How do quantity sold and profit margins relate?**

In [ ]:
sns.scatterplot(x='Quantity', y='Profit_Margin', data=df)
plt.title('Quantity vs. Profit Margin')
plt.show()

**Are there shipping methods causing losses or delays?**

In [ ]:
df = df.reset_index() 
ship_loss = df.groupby('Ship Mode')['Profit'].sum().sort_values()
ship_delay = (df['Ship Date'] - df['Order Date']).dt.days
df['Shipping Delay'] = ship_delay
shipping_delay_avg = df.groupby('Ship Mode')['Shipping Delay'].mean()

In [ ]:
plt.figure(figsize=(16, 6))

#Profit by Ship Mode 
plt.subplot(1, 2, 1)
ax1 = sns.barplot(x=ship_loss.values, y=ship_loss.index, palette='pastel')
ax1.set_title('Total Profit by Shipping Mode')
ax1.set_xlabel('Total Profit')
ax1.set_ylabel('Shipping Mode')
ax1.xaxis.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
#Avg Shipping Delay by Ship Mode
plt.subplot(1, 2, 2)
ax2 = sns.barplot(x=shipping_delay_avg.values, y=shipping_delay_avg.index, palette='pastel')
ax2.set_title('Average Shipping Delay by Mode (in days)')
ax2.set_xlabel('Average Delay (Days)')
ax2.set_ylabel('')
ax2.xaxis.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

**Is the company losing money in certain categories or regions?**

In [ ]:
loss_by_category = df.groupby('Category')['Profit'].sum().sort_values()
loss_by_region = df.groupby('Region')['Profit'].sum().sort_values()


In [ ]:
 
plt.figure(figsize=(16, 6))

#Profit by Category 
plt.subplot(1, 2, 1)
ax1 = sns.barplot(x=loss_by_category.values, y=loss_by_category.index, palette='pastel')
ax1.set_title('Total Profit by Category')
ax1.set_xlabel('Total Profit')
ax1.set_ylabel('Category')
ax1.xaxis.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
#Profit by Region 
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 2)
ax2 = sns.barplot(x=loss_by_region.values, y=loss_by_region.index, palette='pastel')
ax2.set_title('Total Profit by Region')
ax2.set_xlabel('Total Profit')
ax2.set_ylabel('Region')
ax2.xaxis.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

**Can the company forecast sales trends for better planning?**

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

df.set_index('Order Date', inplace=True)
monthly_sales_ts = df['Sales'].resample('M').sum()

decomposition = seasonal_decompose(monthly_sales_ts, model='additive')
decomposition.plot()
plt.show()
